In [1]:
import os
import glob
import pandas as pd
import matplotlib
import numpy as np
from numpy import NaN
import matplotlib.pyplot as plt

In [2]:
# Combine all the CSV files
path = r'../data'
all_csv = glob.glob(path + "/*.csv")

temp = []
for csv in all_csv:
    df = pd.read_csv(csv, index_col=0, header=0, thousands=',')
    temp.append(df)

df = pd.concat(temp, axis=0, ignore_index=True)
df.head()

,Service,Type,Rating,Hires,Price,ZipCode,Zip Code
0,Second Growth Homes,123,NaN,NaN,NaN,99801.0,NaN
1,Colony Construction,123,4.5,27.0,NaN,95814.0,NaN
2,WDS Remodeling & Renovation,123,5.0,18.0,NaN,95814.0,NaN
3,Rose Remodeling,123,5.0,NaN,NaN,95814.0,NaN
4,Lunas handyman.,123,5.0,5.0,NaN,95814.0,NaN


In [ ]:
# Check and fix dtype
df.dtypes

In [ ]:
# See some prices values
df['Price'].unique()[:19]

# Need to fix that hour thing

In [ ]:
# Convert price to string
df['Price'] = df['Price'].astype(str)

# Removed the hour into another column
tempPrice = []
tempPriceType = []
for i in df['Price']:
    x = i.split('/')
    tempPrice.append(x[0])
    if i.find('hour') > 0:
        x = 'hourly'
    else:
        x = NaN
    tempPriceType.append(x)

# Add it to dataframe and check
df['Price'] = tempPrice
df['Price Type'] = tempPriceType

# Convert price column to float
df['Price'] = df['Price'].astype(float)

In [ ]:
# Double check data types
df.dtypes

In [ ]:
# Add the service type from database
serviceDf = pd.read_csv('../src/all_services.csv')

dfMerge = df.rename(columns={"Type":"ID"})
dfMerge = pd.merge(dfMerge, serviceDf, left_on='ID', right_on='ID', how='left').drop('ID', axis=1)
df = dfMerge

In [ ]:
# See shape
df.shape

In [ ]:
# Check for duplicates
dfDuplicate = df[df.duplicated()]
dfDuplicate

# Deal with duplicates accordingly if there are any

In [ ]:
# See overview of data
df.describe()

In [ ]:
# See basic count
servicesCount = df.groupby("Services")["Services"].count().to_frame(name="Count").reset_index()
servicesCount.sort_values(by="Count", ascending=False)

In [ ]:
# Get implied revenue
df["Implied Revenue"] = df["Price"] * df["Hires"]

# Get sum of implied revenue by segment
dfRevenuBySegment = df.groupby("Segments")["Implied Revenue"].sum().to_frame(name="Implied Revenue").reset_index()
dfRevenuBySegment["Contribution"] = dfRevenuBySegment["Implied Revenue"] / dfRevenuBySegment["Implied Revenue"].sum()

# Get sum of implied revenue by services
dfRevenuByServices = df.groupby("Services")["Implied Revenue"].sum().to_frame(name="Implied Revenue").reset_index()
dfRevenuByServices["Contribution"] = dfRevenuByServices["Implied Revenue"] / dfRevenuByServices["Implied Revenue"].sum()

In [ ]:
dfRevenuBySegment.sort_values(by="Contribution", ascending=False)

In [ ]:
dfRevenuByServices.sort_values(by="Contribution", ascending=False)

In [ ]:
df.to_csv("all_services.csv")